In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report
import os
from tensorflow.keras.models import load_model

path = input("Enter the path of the WELFake_Dataset.csv")
df = pd.read_csv(path)


print(df.head())
print(df.info())
print(df['label'].value_counts())



df['combined_text'] = df['title'] + " " + df['text']
df['combined_text'] = df['combined_text'].fillna('')

# df = df.dropna(subset=['combined_text'])

def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

df['cleaned_text'] = df['combined_text'].apply(clean_text)


X = df['cleaned_text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')


X_train_tfidf = vectorizer.fit_transform(X_train)


X_test_tfidf = vectorizer.transform(X_test)


def create_model():

    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train_tfidf.shape[1],)),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])


    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


    model.summary()


    X_train_dense = X_train_tfidf.toarray()
    X_test_dense = X_test_tfidf.toarray()


    history = model.fit(
        X_train_dense, y_train,
        epochs=6,
        batch_size=64,
        validation_split=0.2
    )

    loss, accuracy = model.evaluate(X_test_dense, y_test)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

    model.save("fakenews.h5")
    return X_train_dense, X_test_dense

if not os.path.exists("fakenews.h5"):
    X_train_dense, X_test_dense = create_model()
else:
    model = load_model('fakenews.h5')

y_pred = model.predict(X_test_dense)
y_pred = (y_pred > 0.5).astype(int)



print(classification_report(y_test, y_pred))
def predict_fake_news(text):
    cleaned_text = clean_text(text)
    text_tfidf = vectorizer.transform([cleaned_text])
    text_dense = text_tfidf.toarray()
    prediction = model.predict(text_dense)
    confidence = prediction[0][0]
    label = "Fake" if prediction > 0.5 else "Real"
    return label, confidence

#TEST
# new_article = "This is a fake news article designed to mislead people."
# label, confidence = predict_fake_news(new_article)
# print(f"Prediction: {label} (Confidence: {confidence:.2f})")





   Unnamed: 0                                              title  \
0           0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1           1                                                NaN   
2           2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3           3  Bobby Jindal, raised Hindu, uses story of Chri...   
4           4  SATAN 2: Russia unvelis an image of its terrif...   

                                                text  label  
0  No comment is expected from Barack Obama Membe...      1  
1     Did they post their votes for Hillary already?      1  
2   Now, most of the demonstrators gathered last ...      1  
3  A dozen politically active pastors came here f...      0  
4  The RS-28 Sarmat missile, dubbed Satan 2, will...      1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72134 non-null

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │         640,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 648,449 (2.47 MB)

 Trainable params: 648,449 (2.47 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/6
722/722 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.8671 - loss: 0.3261 - val_accuracy: 0.9544 - val_loss: 0.1265
Epoch 2/6
722/722 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9680 - loss: 0.0932 - val_accuracy: 0.9563 - val_loss: 0.1283
Epoch 3/6
722/722 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9823 - loss: 0.0555 - val_accuracy: 0.9590 - val_loss: 0.1344
Epoch 4/6
722/722 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9887 - loss: 0.0372 - val_accuracy: 0.9591 - val_loss: 0.1486
Epoch 5/6
722/722 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9923 - loss: 0.0249 - val_accuracy: 0.9586 - val_loss: 0.1738
Epoch 6/6
722/722 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9943 - loss: 0.0189 - val_accuracy: 0.9603 - val_loss: 0.1779
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9642 - loss: 0.1482
Test Accuracy: 96.22%
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0       0.96      0.96      0.96  

In [2]:
import math
import requests
from bs4 import BeautifulSoup

def get_article_text(url):
    """Extracts and cleans main article text from a given URL."""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Try extracting main article content
    article = soup.find("article")
    if article:
        paragraphs = article.find_all("p")
    else:
        # If no <article> tag is found, fallback to all <p> tags
        paragraphs = soup.find_all("p")

    text = " ".join(p.get_text() for p in paragraphs if p.get_text().strip())

    return text if text else None  # Return None if no meaningful text is found

while True:
    x = input("If you would like to check text from copy-paste, enter 1 \nIf you would like to check from a website link, enter 2.\nIf you are done, press 0 to exit. ")

    if x not in {"1", "2", "0"}:
        print("Invalid input.")
        continue

    if x == "1":
        new_article = input("Enter your article here: ")
        label, confidence = predict_fake_news(new_article)
        confidence = math.ceil(confidence * 10**5) / 10**5
        print(f"This is most likely {label.lower()} news, and I am {confidence * 100}% sure.")

    elif x == "2":
        url = input("Enter the website link here or press 0 to quit altogether: ")
        if url == "0":
            break

        article_text = get_article_text(url)
        if not article_text:
            print("Could not extract meaningful text from the URL. Try another website.")
            continue

        label, confidence = predict_fake_news(article_text)
        confidence = math.ceil(confidence * 10**5) / 10**5
        print(f"This is most likely {label.lower()} news")

    elif x == "0":
        break

If you would like to check text from copy-paste, enter 1 
If you would like to check from a website link, enter 2.
If you are done, press 0 to exit. 2
Enter the website link here or press 0 to quit altogether: https://www.cnn.com/2025/02/21/politics/trump-fires-top-us-general-cq-brown/index.html
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
This is most likely real news
If you would like to check text from copy-paste, enter 1 
If you would like to check from a website link, enter 2.
If you are done, press 0 to exit. 0
